In [24]:
import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx

# reading input log file
# import ruamel.yaml
import yaml
import re
import ast

import math
import sys
import traceback
from csv import excel
from collections import Counter
from scipy import stats

stat_path="../example_stat/1p9f9s_run"
iamge_path=f"{stat_path}/images"

test_name = "1p9f9s_run"


In [25]:
import os
import re

def search_files_with_name(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if re.search(pattern, file):
                file_list.append(os.path.join(root, file))
                #print(os.path.join(root, file))
    return file_list

In [26]:
vfd_files = search_files_with_name(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(vfd_files)


def load_vfd_yaml(vfd_files):
    ret_dict = {}
    tmp_dict = {}
    for f in vfd_files:
        if "167838" in f: # TODO: now only 7 stages
            continue
        else:
            with open(f, "r") as stream:
                print(f"loading {f}")
                try:
                    tmp_dict = yaml.safe_load(stream)
                    # print(tmp_dict)
                except yaml.YAMLError as exc:
                    print(exc)
                ret_dict[f] = tmp_dict
    return ret_dict

vfd_dict = load_vfd_yaml(vfd_files)
# print(vfd_dict)

print("loading yaml done")

['../example_stat/1p9f9s_run/5904_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/170958_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/75190_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/168881_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/167838_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/98522_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/11179_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/157121_vfd-data-stat-dl.yaml', '../example_stat/1p9f9s_run/77100_vfd-data-stat-dl.yaml']
loading ../example_stat/1p9f9s_run/5904_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/170958_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/75190_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/168881_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/98522_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/11179_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_run/157121_vfd-data-stat-dl.yaml
loading ../example_stat/1p9f9s_

In [27]:
# op_range_shift_dict = {
#     "75190":0,
#     "5904":7208,
#     "77100":12689,
#     "168881":15641,
#     "98522":30542,
#     "170958":31599,
#     "11179":34399,
#     "157121":37801,
#     # "167838":91681,
# }

# task_pid_dict = {
#     "run_idfeature": {"task_pid":75190},
#     "run_tracksingle": {"task_pid":5904},
#     "run_gettracks": {"task_pid":77100},
#     "run_trackstats": {"task_pid":168881},
#     "run_identifymcs": {"task_pid":98522},
#     "run_matchpf": {"task_pid":170958},
#     "run_mcsstats": {"task_pid":11179},
#     "run_robustmcs": {"task_pid":157121},
#     # "run_mapfeature": {"task_pid":167838},
#     # "run_speed": {"task_pid":??},
# }



# prefetch_entry_list = []

# # get the I/O range
# for k,stat in vfd_dict.items():
#     # print(f"VFD Stat File: {k}")
#     task_max_io = 0
#     prev_file_max_io = 0
#     file_op_range_dict = {}
#     for li in stat:
#         file_key = list(li.keys())[0]
#         if 'file' in file_key:
#             # file_stat = list(li.values())[0]
#             file_stat = li[file_key]
#             file_name = file_stat['file_name']
#             file_max_io = 0
#             file_min_io = prev_file_max_io
#             file_max_blob = 0
#             prefetch_type = 0 # 0: no prefetch, 1: prefetch blobs
#             try:
#                 h5mDraw = file_stat['data']['H5FD_MEM_DRAW']
                
#                 if h5mDraw['read_cnt'] > 0:
#                     # print(f"blobs here: {[val[1] for val in h5mDraw['read_ranges'].values()]}")
#                     max_blob = max([val[1] for val in h5mDraw['read_ranges'].values()])
#                     max_io = max(h5mDraw['read_ranges'].keys())
#                     min_io = min(h5mDraw['read_ranges'].keys())
#                     if max_blob > file_max_blob: file_max_blob = max_blob
#                     if max_io > task_max_io: task_max_io = max_io
#                     if max_io > file_max_io: file_max_io = max_io
#                     if min_io < file_min_io: file_min_io = min_io
#                     prefetch_type = 1
                    
#                 elif h5mDraw['write_cnt'] > 0:
#                     # print(f"blobs here: {[val[1] for val in h5mDraw['write_ranges'].values()]}")
#                     max_blob = max([val[1] for val in h5mDraw['write_ranges'].values()])
#                     max_io = max(h5mDraw['write_ranges'].keys())
#                     min_io = min(h5mDraw['read_ranges'].keys())
#                     if max_blob > file_max_blob: file_max_blob = max_blob
#                     if max_io > task_max_io: task_max_io = max_io
#                     if max_io > file_max_io: file_max_io = max_io
#                     if min_io < file_min_io: file_min_io = min_io
#                     prefetch_type = 0
#                 else:
#                     pass
                
#                 for meta_type, meta_stats in file_stat['metadata'].items():
#                     # print(f"meta_type: {meta_type}")
#                     if file_stat['metadata'][meta_type]['read_cnt'] > 0:
#                         max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['read_ranges'].values()])
#                         max_io = max(file_stat['metadata'][meta_type]['read_ranges'].keys())
#                         min_io = min(file_stat['metadata'][meta_type]['read_ranges'].keys())
#                         if max_blob > file_max_blob: file_max_blob = max_blob
#                         if max_io > task_max_io: task_max_io = max_io
#                         if max_io > file_max_io: file_max_io = max_io
#                         if min_io < file_min_io: file_min_io = min_io
#                         prefetch_type = 1
                        
#                     elif file_stat['metadata'][meta_type]['write_cnt'] > 0:
#                         max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['write_ranges'].values()])
#                         max_io = max(file_stat['metadata'][meta_type]['write_ranges'].keys())
#                         min_io = min(file_stat['metadata'][meta_type]['write_ranges'].keys())
#                         if max_blob > file_max_blob: file_max_blob = max_blob
#                         if max_io > task_max_io: task_max_io = max_io
#                         if max_io > file_max_io: file_max_io = max_io
#                         if min_io < file_min_io: file_min_io = min_io
#                         prefetch_type = 0
#                     else:
#                         pass
                
#                 # if "cloudid_20150506_040000" in file_name:
#                 #     print(f"VFD Stat File: {k}")
#                 #     print(f"file_name: {file_name}")
#                 #     print(f"file_io_range: [{file_min_io},{file_max_io}]")
#                 if file_name in file_op_range_dict.keys():
#                     file_op_range_dict[file_name]['io_ranges'] .append((file_min_io, file_max_io))
#                     if file_max_blob != file_op_range_dict[file_name]['blob_reanges'][1]:
#                         file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
#                         file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
                        
#                 else:
#                     file_op_range_dict[file_name] = {}
#                     file_op_range_dict[file_name]['io_ranges'] = [(file_min_io, file_max_io)]
#                     file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
#                     file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
            
#                 prev_file_max_io = file_max_io
#                 if prefetch_type == 0:
#                     print(f"file_op_range_dict: {file_op_range_dict[file_name]}")

#             except:
#                 pass


#     file_name = k.split('/')[-1]
#     task_id = file_name.split('_')[0]
#     # print(f"{task_id} Max IO IDX: {task_max_io}")
#     range_shift = op_range_shift_dict[task_id]
#     page_size_factor = 1 # comparing to 1M, 512KB is 2 for 1M
    
#     for file_name, ranges_val in file_op_range_dict.items():
#         for range in ranges_val['io_ranges']:
#             pre_entry = {}
#             # pre_entry['op_count_range'] = f"[{page_size_factor*(range_shift+0)}, {page_size_factor*(range_shift+task_max_io)}]"
#             # pre_entry['op_count_range'] = [page_size_factor*(range_shift+range[0]), page_size_factor*(range_shift+range[1])]
#             pre_entry['op_count_range'] = [page_size_factor*(range_shift), page_size_factor*(range_shift)]


#             pre_entry['prefetch'] = []
#             bucket_entry = {}
#             bucket_entry['bucket'] = file_name
#             # bucket_entry['promote_blobs'] = f"[{page_size_factor*(ranges_val['blob_reanges'][0])}, {page_size_factor*(ranges_val['blob_reanges'][1])}]"
#             bucket_entry['blob_ranges'] = [page_size_factor*(ranges_val['blob_reanges'][0]), page_size_factor*(ranges_val['blob_reanges'][1])]
#             bucket_entry['prefetch_type'] = "promote_blobs" if ranges_val['prefetch_type'] == 1 else "demote_blobs"
            
#             pre_entry['prefetch'].append(bucket_entry)
#             prefetch_entry_list.append(pre_entry)
            

# with open('./apriori_p-1p9f-512KB.yml', 'w') as outfile:
#     outfile.writelines("0:\n")
#     sorted_entries = sorted(prefetch_entry_list, key=lambda x: x['op_count_range'][0])

#     for ent in sorted_entries:
#         # print(f"  - op_count_range: {ent['op_count_range']}")
#         # print(f"    prefetch:")
        
#         outfile.writelines(f"  - op_count_range: {ent['op_count_range']}\n")
#         outfile.writelines(f"    prefetch:\n")
#         for bucket in ent['prefetch']:
#             # print(f"      - bucket: \"{bucket['bucket']}\"")
#             # print(f"        promote_blobs: {bucket['promote_blobs']}")
#             # TODO: bucketname correction
#             prefix = bucket['bucket'].split('_')[0]
#             if prefix == "cloudid" or prefix == "track":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/tracking"
#             if prefix == "mcs" or prefix == "trackstats":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/stats"
#             if prefix == "wrfout" or prefix == "wrf":
#                 prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_input_data/wrf_tbradar"
#             bucket_name = prefix_path + "/" + bucket['bucket']
#             outfile.writelines(f"      - bucket: \"{bucket_name}\"\n")
#             outfile.writelines(f"        {bucket['prefetch_type']}: {bucket['blob_ranges']}\n")

    

In [30]:
op_range_shift_dict = {
    "75190":0,
    "5904":7208,
    "77100":12689,
    "168881":15641,
    "98522":30542,
    "170958":31599,
    "11179":34399,
    "157121":37801,
    # "167838":91681,
}

task_pid_dict = {
    "run_idfeature": {"task_pid":75190},
    "run_tracksingle": {"task_pid":5904},
    "run_gettracks": {"task_pid":77100},
    "run_trackstats": {"task_pid":168881},
    "run_identifymcs": {"task_pid":98522},
    "run_matchpf": {"task_pid":170958},
    "run_mcsstats": {"task_pid":11179},
    "run_robustmcs": {"task_pid":157121},
    # "run_mapfeature": {"task_pid":167838},
    # "run_speed": {"task_pid":??},
}



prefetch_entry_list = []

# get the I/O range
for k,stat in vfd_dict.items():
    # print(f"VFD Stat File: {k}")
    task_max_io = 0
    prev_file_max_io = 0
    file_op_range_dict = {}
    for li in stat:
        file_key = list(li.keys())[0]
        if 'file' in file_key:
            # file_stat = list(li.values())[0]
            file_stat = li[file_key]
            file_name = file_stat['file_name']
            file_max_io = 0
            file_min_io = prev_file_max_io
            file_max_blob = 0
            prefetch_type = 0 # 0: no prefetch, 1: prefetch blobs
            try:
                h5mDraw = file_stat['data']['H5FD_MEM_DRAW']
                
                if h5mDraw['read_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['read_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['read_ranges'].values()])
                    max_io = max(h5mDraw['read_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                    prefetch_type = 1
                    
                elif h5mDraw['write_cnt'] > 0:
                    # print(f"blobs here: {[val[1] for val in h5mDraw['write_ranges'].values()]}")
                    max_blob = max([val[1] for val in h5mDraw['write_ranges'].values()])
                    max_io = max(h5mDraw['write_ranges'].keys())
                    min_io = min(h5mDraw['read_ranges'].keys())
                    if max_blob > file_max_blob: file_max_blob = max_blob
                    if max_io > task_max_io: task_max_io = max_io
                    if max_io > file_max_io: file_max_io = max_io
                    if min_io < file_min_io: file_min_io = min_io
                    prefetch_type = 0
                else:
                    pass
                
                for meta_type, meta_stats in file_stat['metadata'].items():
                    # print(f"meta_type: {meta_type}")
                    if file_stat['metadata'][meta_type]['read_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['read_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['read_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                        prefetch_type = 1
                        
                    elif file_stat['metadata'][meta_type]['write_cnt'] > 0:
                        max_blob = max([val[1] for val in file_stat['metadata'][meta_type]['write_ranges'].values()])
                        max_io = max(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        min_io = min(file_stat['metadata'][meta_type]['write_ranges'].keys())
                        if max_blob > file_max_blob: file_max_blob = max_blob
                        if max_io > task_max_io: task_max_io = max_io
                        if max_io > file_max_io: file_max_io = max_io
                        if min_io < file_min_io: file_min_io = min_io
                        prefetch_type = 0
                    else:
                        pass
                
                # if "cloudid_20150506_040000" in file_name:
                #     print(f"VFD Stat File: {k}")
                #     print(f"file_name: {file_name}")
                #     print(f"file_io_range: [{file_min_io},{file_max_io}]")
                if file_name in file_op_range_dict.keys():
                    file_op_range_dict[file_name]['io_ranges'] .append((file_min_io, file_max_io))
                    if file_max_blob != file_op_range_dict[file_name]['blob_reanges'][1]:
                        file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                        file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
                        
                else:
                    file_op_range_dict[file_name] = {}
                    file_op_range_dict[file_name]['io_ranges'] = [(file_min_io, file_max_io)]
                    file_op_range_dict[file_name]['blob_reanges'] = (0, file_max_blob)
                    file_op_range_dict[file_name]['prefetch_type'] = prefetch_type
            
                prev_file_max_io = file_max_io
                if prefetch_type == 0:
                    print(f"file_op_range_dict: {file_op_range_dict[file_name]}")

            except:
                pass


    file_name = k.split('/')[-1]
    task_id = file_name.split('_')[0]
    # print(f"{task_id} Max IO IDX: {task_max_io}")
    range_shift = op_range_shift_dict[task_id]
    page_size_factor = 1 # comparing to 1M, 512KB is 2 for 1M
    
    for file_name, ranges_val in file_op_range_dict.items():
        for range in ranges_val['io_ranges']:
            pre_entry = {}
            # pre_entry['op_count_range'] = f"[{page_size_factor*(range_shift+0)}, {page_size_factor*(range_shift+task_max_io)}]"
            # pre_entry['op_count_range'] = [page_size_factor*(range_shift+range[0]), page_size_factor*(range_shift+range[1])]
            pre_entry['op_count_range'] = [page_size_factor*(range_shift), page_size_factor*(range_shift)]


            pre_entry['prefetch'] = []
            bucket_entry = {}
            bucket_entry['bucket'] = file_name
            # bucket_entry['promote_blobs'] = f"[{page_size_factor*(ranges_val['blob_reanges'][0])}, {page_size_factor*(ranges_val['blob_reanges'][1])}]"
            bucket_entry['blob_ranges'] = [page_size_factor*(page_size_factor*ranges_val['blob_reanges'][0]), page_size_factor*(page_size_factor*ranges_val['blob_reanges'][1])]
            bucket_entry['prefetch_type'] = "promote_blobs" if ranges_val['prefetch_type'] == 1 else "demote_blobs"
            
            pre_entry['prefetch'].append(bucket_entry)
            prefetch_entry_list.append(pre_entry)
            

with open('./apriori_p-1p9f-512KB.yml', 'w') as outfile:
    outfile.writelines("0:\n")
    sorted_entries = sorted(prefetch_entry_list, key=lambda x: x['op_count_range'][0])

    for ent in sorted_entries:
        # print(f"  - op_count_range: {ent['op_count_range']}")
        # print(f"    prefetch:")
        
        outfile.writelines(f"  - op_count_range: {ent['op_count_range']}\n")
        outfile.writelines(f"    prefetch:\n")
        for bucket in ent['prefetch']:
            # print(f"      - bucket: \"{bucket['bucket']}\"")
            # print(f"        promote_blobs: {bucket['promote_blobs']}")
            # TODO: bucketname correction
            prefix = bucket['bucket'].split('_')[0]
            if prefix == "cloudid" or prefix == "track":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/tracking"
            if prefix == "mcs" or prefix == "trackstats":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_wrf_tbradar/stats"
            if prefix == "wrfout" or prefix == "wrf":
                prefix_path = "/qfs/projects/oddite/tang584/flextrkr_runs/hm_input_data/wrf_tbradar"
            bucket_name = prefix_path + "/" + bucket['bucket']
            outfile.writelines(f"      - bucket: \"{bucket_name}\"\n")
            outfile.writelines(f"        {bucket['prefetch_type']}: {bucket['blob_ranges']}\n")
